In [1]:
!pip install -U qiskit qiskit-aer qiskit-machine-learning qiskit-algorithms scikit-learn


  Using cached qiskit-2.2.3-cp39-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (12 kB)


In [2]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

from qiskit.primitives import Sampler
from qiskit.circuit.library import ZZFeatureMap, TwoLocal
from qiskit_machine_learning.algorithms import VQC
from qiskit_algorithms.optimizers import COBYLA


In [3]:
data = pd.read_csv('/content/sonar.csv', header=None)

X = data.iloc[:, :-1]
y = data.iloc[:, -1].map({'R': 0, 'M': 1})


In [4]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=4)   # engineer-level (not oversimplified)
X_reduced = pca.fit_transform(X_scaled)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X_reduced, y, test_size=0.2, stratify=y, random_state=42
)

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()


In [6]:
baseline = LogisticRegression(max_iter=1000)
baseline.fit(X_train, y_train)

baseline_pred = baseline.predict(X_test)
baseline_acc = accuracy_score(y_test, baseline_pred)

print("Classical Baseline Accuracy:", baseline_acc)


Classical Baseline Accuracy: 0.7142857142857143


In [7]:
X_train_q = X_train[:, :2]
X_test_q = X_test[:, :2]


In [8]:
feature_map = ZZFeatureMap(
    feature_dimension=2,
    reps=2,
    entanglement='linear'
)

ansatz = TwoLocal(
    num_qubits=2,
    rotation_blocks='ry',
    entanglement_blocks='cz',
    reps=2
)


In [22]:
from qiskit.primitives import StatevectorSampler
sampler = StatevectorSampler()


In [10]:
vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer,
    sampler=sampler
)


In [11]:
vqc.fit(X_train_q, y_train)


In [12]:
q_pred = vqc.predict(X_test_q)
q_acc = accuracy_score(y_test, q_pred)

print("Quantum VQC Accuracy:", q_acc)


Quantum VQC Accuracy: 0.5714285714285714


In [13]:
probs = vqc.predict_proba(X_test_q)

sample_id = 0
confidence = np.max(probs[sample_id]) * 100
prediction = q_pred[sample_id]

print("Prediction:", "MINE" if prediction == 1 else "ROCK")
print(f"Confidence: {confidence:.2f}%")


Prediction: ROCK
Confidence: 91.20%


In [14]:
final_prediction = (
    q_pred if q_acc > baseline_acc else baseline_pred
)

final_acc = accuracy_score(y_test, final_prediction)
print("Final Hybrid System Accuracy:", final_acc)


Final Hybrid System Accuracy: 0.7142857142857143


In [15]:
sample_id = 3

prediction = q_pred[sample_id]
confidence = np.max(probs[sample_id]) * 100

print("Prediction:", "MINE" if prediction == 1 else "ROCK")
print(f"Confidence: {confidence:.2f}%")


Prediction: MINE
Confidence: 56.48%


In [17]:

preds = vqc.predict(X_test_q[:5])
probs = vqc.predict_proba(X_test_q[:5])

for i in range(5):
    label = "MINE" if preds[i] == 1 else "ROCK"
    conf = np.max(probs[i]) * 100
    print(f"Sample {i}: {label} ({conf:.2f}%)")


Sample 0: ROCK (91.20%)
Sample 1: MINE (64.45%)
Sample 2: MINE (65.62%)
Sample 3: MINE (56.48%)
Sample 4: ROCK (73.15%)
